# User groups

In [15]:
request = "POST https://analyticsreporting.googleapis.com/v4/reports:batchGet?fields=reports(columnHeader%2Cdata(rows%2Ctotals))&key={YOUR_API_KEY}"
request = {
 "reportRequests": [
  {
   "viewId": "123303369",
   "dateRanges": [
    {
     "startDate": "2017-01-01",
     "endDate": "2017-04-30"
    }
   ],
   "metrics": [
    {
     "expression": "ga:sessions"
    },
    {
     "expression": "ga:sessionDuration"
    },
    {
     "expression": "ga:goal1Completions"
    },
    {
     "expression": "ga:bounceRate"
    }
   ],
   "dimensions": [
    {
     "name": "ga:city"
    },
    {
     "name": "ga:userAgeBracket"
    }
   ]
  }
 ]
}

In [139]:
import json

with open('data/TMRW_users.json') as file:
    input_ugroups = json.load(file)
    
#input_ugroups

# Define dimensions list
input_ugroups_dimensions = input_ugroups['reports'][0]['columnHeader']['dimensions']

dimension_count = len(input_ugroups_dimensions)

# Define metrics list
input_ugroups_metrics = input_ugroups['reports'][0]['columnHeader']['metricHeader']['metricHeaderEntries']

def create_metric_list(raw_data):
    lst = []
    for item in raw_data:
        lst.append(item['name'])
    return lst

input_ugroups_metrics = create_metric_list(input_ugroups_metrics)


# Create input data

input_ugroups_data = input_ugroups['reports'][0]['data']['rows']

input_ugroups_data

[{'dimensions': ['(not set)', '25-34'],
  'metrics': [{'values': ['27', '1848.0', '1', '62.96296296296296']}]},
 {'dimensions': ['(not set)', '35-44'],
  'metrics': [{'values': ['23', '1790.0', '1', '52.17391304347826']}]},
 {'dimensions': ['Croydon', '18-24'],
  'metrics': [{'values': ['101', '10047.0', '4', '41.584158415841586']}]},
 {'dimensions': ['Croydon', '25-34'],
  'metrics': [{'values': ['334', '62737.0', '17', '47.90419161676647']}]},
 {'dimensions': ['Croydon', '35-44'],
  'metrics': [{'values': ['223', '34564.0', '7', '43.04932735426009']}]},
 {'dimensions': ['Croydon', '45-54'],
  'metrics': [{'values': ['90', '14131.0', '2', '45.55555555555556']}]},
 {'dimensions': ['Croydon', '55-64'],
  'metrics': [{'values': ['32', '1932.0', '1', '53.125']}]},
 {'dimensions': ['Hove', '35-44'],
  'metrics': [{'values': ['12', '833.0', '1', '16.666666666666664']}]},
 {'dimensions': ['London', '18-24'],
  'metrics': [{'values': ['167', '29335.0', '8', '49.700598802395206']}]},
 {'dimens

In [140]:
values_list = []
for group in input_ugroups_data:
    new_dim_name = group['dimensions'][0] + ", " + group['dimensions'][1]
    group[new_dim_name] = group['metrics'][0]
    del group['dimensions']
    del group['metrics']
    
    
    
    conv_rate = round(float(int(group[new_dim_name]['values'][2])/int(group[new_dim_name]['values'][0])*100),2)
    values_list.append(group[new_dim_name]['values'])
    
    group[new_dim_name]['values'].append(conv_rate)
    
    for value in group[new_dim_name]['values']:
        value = list(map(float, value))
        print(value)
    
    
    
    
    
values_list
#input_ugroups_data

[2.0, 7.0]


ValueError: could not convert string to float: '.'

In [119]:

    for i in 
        results = list(map(float, values_list[0]))
        results

[27.0, 1848.0, 1.0, 62.96296296296296, 3.7]

In [34]:
# Define each metric dict

ugroups_data = {}

for ugroup in input_ugroups_data:
    #print (ugroup)
    
    for gr in ugroup:
        ugroups_data[gr] = {'sessions':0,
                       'duration':0,
                       'bounce rate':0,
                       'conversions':0,
                       'conversion rate':0}
        
        ugroups_data[gr]['sessions'] = int(ugroup[gr]['values'][0])
        ugroups_data[gr]['duration'] = float(ugroup[gr]['values'][1])
        ugroups_data[gr]['conversions'] = int(ugroup[gr]['values'][2])
        ugroups_data[gr]['bounce rate'] = float(ugroup[gr]['values'][3])
        ugroups_data[gr]['conversion rate'] = float(ugroup[gr]['values'][4])
        
#ugroups_data

In [35]:
rows = list(ugroups_data.keys())
rows

['(not set), 25-34',
 '(not set), 35-44',
 'Croydon, 18-24',
 'Croydon, 25-34',
 'Croydon, 35-44',
 'Croydon, 45-54',
 'Croydon, 55-64',
 'Hove, 35-44',
 'London, 18-24',
 'London, 25-34',
 'London, 35-44',
 'London, 45-54',
 'London, 55-64',
 'London, 65+']

In [36]:
import collections
from collections import OrderedDict

columns = []
for u in ugroups_data:
    #print (test[r])
    for metric in ugroups_data[u]:
        columns.append(metric)
    
columns = list(OrderedDict.fromkeys(columns))    
columns

['sessions', 'duration', 'bounce rate', 'conversions', 'conversion rate']

In [37]:
import pandas as pd

In [39]:
df = pd.DataFrame(values_list,
                  index = rows,
                  columns = columns)

df.to_json(orient='split')
pd.read_json(df.to_json(orient='split'), orient='split')

,sessions,duration,bounce rate,conversions,conversion rate
"(not set), 25-34",27,1848,1,62.962963,3.703704
"(not set), 35-44",23,1790,1,52.173913,4.347826
"Croydon, 18-24",101,10047,4,41.584158,3.960396
"Croydon, 25-34",334,62737,17,47.904192,5.089820
"Croydon, 35-44",223,34564,7,43.049327,3.139013
"Croydon, 45-54",90,14131,2,45.555556,2.222222
"Croydon, 55-64",32,1932,1,53.125000,3.125000
"Hove, 35-44",12,833,1,16.666667,8.333333
"London, 18-24",167,29335,8,49.700599,4.790419
"London, 25-34",842,84813,24,59.144893,2.850356


In [94]:
df['sessions'].sum()

'27231013342239032121678424822053715'

In [47]:
import numpy as np
 
def cluster_points(X, mu):
    clusters  = {}
    for x in X:
        bestmukey = min([(i[0], np.linalg.norm(x-mu[i[0]])) \
                    for i in enumerate(mu)], key=lambda t:t[1])[0]
        try:
            clusters[bestmukey].append(x)
        except KeyError:
            clusters[bestmukey] = [x]
    return clusters
 
cluster_points(values_list, [12, 842])


TypeError: unsupported operand type(s) for -: 'list' and 'int'

In [ ]:
def reevaluate_centers(mu, clusters):
    newmu = []
    keys = sorted(clusters.keys())
    for k in keys:
        newmu.append(np.mean(clusters[k], axis = 0))
    return newmu

def has_converged(mu, oldmu):
    return set([tuple(a) for a in mu]) == set([tuple(a) for a in oldmu])

def find_centers(X, K):
    # Initialize to K random centers
    oldmu = random.sample(X, K)
    mu = random.sample(X, K)
    while not has_converged(mu, oldmu):
        oldmu = mu
        # Assign all points in X to clusters
        clusters = cluster_points(X, mu)
        # Reevaluate centers
        mu = reevaluate_centers(oldmu, clusters)
    return(mu, clusters)

In [41]:
import random
 
def init_board(N):
    X = np.array([(random.uniform(-1, 1), random.uniform(-1, 1)) for i in range(N)])
    return X

In [42]:
def init_board_gauss(N, k):
    n = float(N)/k
    X = []
    for i in range(k):
        c = (random.uniform(-1, 1), random.uniform(-1, 1))
        s = random.uniform(0.05,0.5)
        x = []
        while len(x) < n:
            a, b = np.array([np.random.normal(c[0], s), np.random.normal(c[1], s)])
            # Continue drawing points from the distribution in the range [-1,1]
            if abs(a) < 1 and abs(b) < 1:
                x.append([a,b])
        X.extend(x)
    X = np.array(X)[:N]
    return X

In [14]:
import d3py
import pandas
import numpy as np
from d3py import pandas_figure

# some test data
T = 100
# this is a data frame with three columns (we only use 2)
df = pd.DataFrame({
    "time" : range(T),
    "pressure": np.random.rand(T),
    "temp" : np.random.rand(T)
})
## build up a figure, ggplot2 style
# instantiate the figure object
fig = d3py.PandasFigure(df, name="basic_example", width=300, height=300) 
# add some red points
fig += d3py.geoms.Point(x="pressure", y="temp", fill="red")
# writes 3 files, starts up a server, then draws some beautiful points in Chrome
fig.show() 

ModuleNotFoundError: No module named 'pandas_figure'

In [80]:
import numpy as np

In [81]:
# Function: K Means
# -------------
# K-Means is an algorithm that takes in a dataset and a constant
# k and returns k centroids (which define clusters of data in the
# dataset which are similar to one another).

def kmeans(dataSet, k):

    # Initialize centroids randomly
    numFeatures = dataSet.getNumFeatures()
    centroids = getRandomCentroids(numFeatures, k)
    
    # Initialize book keeping vars.
    iterations = 0
    oldCentroids = None
    
    # Run the main k-means algorithm
    while not shouldStop(oldCentroids, centroids, iterations):
        # Save old centroids for convergence test. Book keeping.
        oldCentroids = centroids
        iterations += 1
        
        # Assign labels to each datapoint based on centroids
        labels = getLabels(dataSet, centroids)
        
        # Assign centroids based on datapoint labels
        centroids = getCentroids(dataSet, labels, k)
        
    # We can get the labels too by calling getLabels(dataSet, centroids)
    return centroids

In [82]:
# Function: Should Stop
# -------------
# Returns True or False if k-means is done. K-means terminates either
# because it has run a maximum number of iterations OR the centroids
# stop changing.

def shouldStop(oldCentroids, centroids, iterations):
    if iterations > MAX_ITERATIONS: return True
    return oldCentroids == centroids

In [83]:
# Function: Get Labels
# -------------
# Returns a label for each piece of data in the dataset. 


#def getLabels(dataSet, centroids):
    # For each element in the dataset, chose the closest centroid. 
    # Make that centroid the element's label

In [91]:
x = np.array([1, 2, 3, 4, 5])
y = np.array([8, 8, 8, 8, 8])
z = np.ones((5, 9))

In [92]:
z - x

ValueError: operands could not be broadcast together with shapes (5,9) (5,) 

In [85]:
np.sqrt(sum((x - y) ** 2))

11.61895003862225

In [86]:
np.sqrt(((z-x)**2).sum(axis=0))

ValueError: operands could not be broadcast together with shapes (5,9) (5,) 